In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

In [2]:
myAPI = os.environ.get('Census_API')

In [3]:
#variables
year = '2019'
cols = 'EMP,EMP_N,ESTAB'

#NAICS 2019
naics = ['334510','334516','334517','339112','339113','3254','54171','541713','541714',\
         '541715','541380','6215']

#NAICS 2010
naics_10 = ['334510','334516','334517','339112','339113','325411','325412','325413','325414',\
            '54171','541711','541712','541380','621511','621512']

In [4]:
#geographies - 2019
zip_nyc = pd.read_table('31cr_ZCTA.txt')
zip_oth = pd.read_csv('SFSJ_BOS_ZCTA.csv')
zip_df = pd.concat([zip_nyc,zip_oth])
zip_df['ZIPCODE']=zip_df.ZIPCODE.astype(str).str.zfill(5)
zip_codes = zip_df['ZIPCODE'].to_list()

In [5]:
#2010 Zip Codes
zip_df_10 = pd.read_csv('ZCTA_2010.csv')
zip_df_10['ZCTA']=zip_df_10.ZCTA.astype(str).str.zfill(5)
zip_codes_10 = zip_df_10['ZCTA'].to_list()

## 2019 - API

In [6]:
frames = []
for n in naics:
    base_url=f'https://api.census.gov/data/{year}/cbp?'
    url = f'{base_url}get={cols}&for=zip%20code:*&NAICS2017={n}&LFO=001&EMPSZES=001&key={myAPI}'
    resp = requests.request('GET',url).content
    if resp == b'':
        pass
    else:
        df = pd.DataFrame(json.loads(resp)[1:])
        df.columns = json.loads(resp)[0]
        frames.append(df)

dff = pd.concat(frames)

In [7]:
dfff = dff[dff['zip code'].isin(zip_codes)]

dfff.to_excel('lifesci_2019_zips.xlsx')

# 2010 - CSV file

In [8]:
df_10 = pd.read_csv('../data/cbp/zbp10detail.txt')

In [9]:
df_10['zip']=df_10.zip.astype(str).str.zfill(5)
df_10 = df_10[(df_10.zip.isin(zip_codes_10))&(df_10.naics.isin(naics_10))]

In [11]:
df_10.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000
794,01010,339112,1,0,0,0,0,0,1,0,0,0
2148,01034,541712,1,1,0,0,0,0,0,0,0,0
2427,01037,541711,1,0,0,1,0,0,0,0,0,0
3849,01069,334516,1,0,0,0,0,1,0,0,0,0
4663,01082,621512,1,1,0,0,0,0,0,0,0,0


In [12]:
df_10.to_excel('lifesci_2010_zips.xlsx')